In [1]:
import pandas as pd
import datetime
import json
import openpyxl
import requests
from fake_useragent import UserAgent

In [2]:
# State ID. Check "State_ID_Mapping.csv" to get the mapping of state id and name.
state_code = 33 #Tripura

# Age of the person to be checked
age = 28

# Number of days ahead of checking availability
numdays = 7

In [3]:
temp_user_agent = UserAgent()
browser_header = {'User-Agent': temp_user_agent.random}

In [4]:
# Getting the district name and their corresponding IDs
dist_IDs = []
dist_Name = []
print(f"Entered State code is: {state_code}")
df_state_id = pd.read_csv("State_ID_Mapping.csv")
state_name = df_state_id[df_state_id["state_id"]==state_code]["state_name"].to_list()[0]
print(f"The corresponding State name is: {state_name}")
response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_code),
                        headers=browser_header)
json_data = json.loads(response.text)
for i in json_data["districts"]:
    dist_IDs.append(i["district_id"])
    dist_Name.append(i["district_name"])

print("District IDs of the State: ", dist_IDs)
print("District Names of the State: ", dist_Name)

Entered State code is: 33
The corresponding State name is: Tripura
District IDs of the State:  [614, 615, 616, 617, 618, 619, 620, 621]
District Names of the State:  ['Dhalai', 'Gomati', 'Khowai', 'North Tripura', 'Sepahijala', 'South Tripura', 'Unakoti', 'West Tripura']


In [5]:
# Getting the dates
base = datetime.datetime.today()
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
date_str = [x.strftime("%d-%m-%Y") for x in date_list]

In [6]:
# Master dataframe for availability list across all regions
cols = ["Date", "District_ID", "District_Name", "Center_Name", "Slots_Available", 
        "Min_Age_Limit", "Fee", "Vaccine_Name", "Address", "Pincode"]
df_avail = pd.DataFrame(columns = cols)

In [7]:
date_list = []
dist_ID_list = []
dist_nm_list = []
center_nm_list = []
avail_list = []
age_list = []
fee_list = []
vaccine_list = []
address_list = []
pincode_list = []

In [8]:
# Getting the available slots
for INP_DATE in date_str:
    for ind, DIST_ID in enumerate(dist_IDs):
        URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, INP_DATE)
        response = requests.get(URL, headers=browser_header)
        print(DIST_ID, INP_DATE)
        if response.ok:
            resp_json = response.json()
            if resp_json["centers"]:
                for center in resp_json["centers"]:
                    for session in center["sessions"]:
                        if session["min_age_limit"] <= age and session["available_capacity"]!=0:
                            date_list.append(INP_DATE)
                            dist_ID_list.append(DIST_ID)
                            dist_nm_list.append(dist_Name[ind])
                            center_nm_list.append(center["name"])
                            avail_list.append(session["available_capacity"])
                            age_list.append(session["min_age_limit"])
                            fee_list.append(center["fee_type"])
                            address_list.append(center["address"])
                            pincode_list.append(center["pincode"])
                            if(session["vaccine"] != ''):
                                vaccine_list.append(session["vaccine"])
                            else:
                                vaccine_list.append("NA")
                        else:
                            print("\t Availability is of different age group or there is no available stock")
                            print("\t Minimum Age Limit: ", session["min_age_limit"])
                            print("\t Available Capacity: ", session["available_capacity"])

            else:
                print("No available slots on {}".format(INP_DATE))

        else:
            print("Invalid response code")
    

614 15-05-2021
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availab

616 15-05-2021
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  5
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  5
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  0
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  26
617 15-05-2021
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  8
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capa

	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  50
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability i

619 15-05-2021
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  99
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  96
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity: 

620 15-05-2021
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  199
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  200
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  199
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  9
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  193
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  1

...

	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
619 21-05-2021
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock


	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  99
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  98
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  100
	 Availability is of different age group or there is no available stock
	 Minimum 

	 Available Capacity:  7
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  9
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  10
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capaci

	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capa

	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability is of different age group or there is no available stock
	 Minimum Age Limit:  45
	 Available Capacity:  40
	 Availability i

In [9]:
df_avail["Date"] = date_list
df_avail["District_ID"] = dist_ID_list
df_avail["District_Name"] = dist_nm_list
df_avail["Center_Name"] = center_nm_list
df_avail["Slots_Available"] = avail_list
df_avail["Min_Age_Limit"] = age_list
df_avail["Fee"] = fee_list
df_avail["Vaccine_Name"] = vaccine_list
df_avail["Address"] = address_list
df_avail["Pincode"] = pincode_list

In [10]:
df_avail.head()

,Date,District_ID,District_Name,Center_Name,Slots_Available,Min_Age_Limit,Fee,Vaccine_Name,Address,Pincode


In [11]:
df_Overview = pd.DataFrame(columns=["Date", "District_Name", "Available_Center_Count", 
                                    "Slots_Available", "Age_Group"])

In [12]:
# Overview sheet generation
date_app = []
place = []
center_count = []
total_avail = []
df_list = []
df_list.append(df_Overview)
dst_list = []
for item in dist_Name:
    df_temp = pd.DataFrame()
    df_temp = df_avail[df_avail["District_Name"] == item]
    if df_temp.shape[0] == 0:
        for dt in date_str:
            date_app.append(dt)
            place.append(item)
            center_count.append(0)
            total_avail.append(0)
    else:
        df_list.append(df_temp)
        dst_list.append(item)
        for dt in date_str:
            date_app.append(dt)
            place.append(item)
            df_temp2 = df_temp[df_temp["Date"] == dt]
            if df_temp2.shape[0] == 0:
                center_count.append(0)
                total_avail.append(0)
            else:
                cnt_lst = df_temp2["Center_Name"].to_list()
                center_count.append(len(cnt_lst))
                slot_cnt = df_temp2["Slots_Available"].to_list()
                total_avail.append(sum(slot_cnt))

In [13]:
df_Overview["Date"] = date_app
df_Overview["District_Name"] = place
df_Overview["Available_Center_Count"] = center_count
df_Overview["Slots_Available"] = total_avail
if age >= 45:
    age_str = '45+'
    df_Overview["Age_Group"] = ['45+']*len(date_app)
elif age >= 18 and age <= 44:
    age_str = '18+'
    df_Overview["Age_Group"] = ['18+']*len(date_app)
#df_Overview["Further_info"] = ["For detailed info on each district, look into the district-wise sheets in same excel"]*len(date_app)

In [14]:
df_list[0] = df_Overview

In [15]:
# Writing the dataframes into different sheets of excel 
file_name = age_str + "_" + state_name + "_Vaccine_Availability.xlsx"
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
for ind, dtframe in enumerate(df_list):
    if ind==0:
        name = "Overview"
    else:
        name = str(dst_list[ind-1]) + "_Availability"
    dtframe.to_excel(writer, name, index=False)
writer.save()
writer.close()

C:\Users\nitha\Anaconda3\envs\v-cowin2\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
